In [1]:
import os

import numpy as np
import tensorflow as tf

from models.model_type import MODELS
from utils.data_utils import DatasetVectorizer
from utils.other_utils import init_config
from utils.other_utils import logger


dirs = ['./Annotation/Saurabh','./Annotation/Rachna + K','./Annotation/Kaveri','./Annotation/Zubair','./Annotation/Saujas']


QUESTIONS = ["qy", "qw", "qo", "qr","qh"]

def get_similarity(sentence1, sentence2):
    return(cosine(model.encode(sentence1)[0], model.encode(sentence2)[0]))
    
    # similarity_processor.get_similarity(sentence1, sentence2)


def is_question(utterance):
    try:
        if utterance.split("|")[2].strip(" ") in QUESTIONS: 
            return 1
    except:
        return 0

/home/saurabh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
class MultiheadSiameseNetGuiDemo:

    def __init__(self):

        self.resultLabel = {'Result' : 0}


        self.main_config = init_config()
        self.model_dir = str(self.main_config['DATA']['model_dir'])

        model_dirs = [os.path.basename(x[0]) for x in os.walk(self.model_dir)]


        self.vectorizer = DatasetVectorizer(self.model_dir)

        self.max_doc_len = self.vectorizer.max_sentence_len
        self.vocabulary_size = self.vectorizer.vocabulary_size

        self.session = tf.Session()
        self.model = self.load_model1("rnn_64")

    def predict(self,sentences1,sentences2):
        if self.model:
            sentence1 = sentences1
            sentence2 = sentences2
            x1_sen = self.vectorizer.vectorize(sentence1)
            x2_sen = self.vectorizer.vectorize(sentence2)
            feed_dict = {self.model.x1: x1_sen, self.model.x2: x2_sen,
                         self.model.is_training: False}
            prediction = np.squeeze(self.session.run([self.model.predictions], feed_dict=feed_dict))
            prediction = np.round(prediction, 2)
            self.resultLabel['text'] = prediction
            return prediction


 

    def load_model(self, model_name):
        logger.info('Loading model: %s', model_name)

        model = MODELS[model_name.split('_')[0]]
        model_config = init_config(model_name.split('_')[0])

        self.model = model(self.max_doc_len, self.vocabulary_size, self.main_config, model_config)
        saver = tf.train.Saver()
        last_checkpoint = tf.train.latest_checkpoint('{}/{}'.format(self.model_dir, model_name))
        saver.restore(self.session, last_checkpoint)
        logger.info('Loaded model from: %s', last_checkpoint)
        saver.save(self.session, 'my_test_model',global_step=1000)
        pos = 0
        count = 0
        for x in dirs: 
            for file in os.listdir(x):
                f = open(x+'/'+file,'r')
                print(f.name)
                lines =f.readlines()
                scan_range = 10
                for index, utterance in enumerate(lines):
                    if index < scan_range or index >= len(lines) - scan_range:
                        continue
                    if is_question(utterance):
                        a1 = []
                        a2 = []
                        for i in range(-scan_range,0):
                            if len((lines[index+i]).split("|")[1]) > 0:
                                a1.append(self.predict(lines[index].split("|")[1], lines[index+i].split("|")[1]))
                        for i in range(1,scan_range+1):
                            if len((lines[index+i]).split("|")[1]) > 0:
                                a2.append(self.predict(lines[index].split("|")[1], lines[index+i].split("|")[1]))
#                         a1 = [x for x in a1 if x > 0.68]
#                         a2 = [x for x in a2 if x > 0.68]
                        keys = ['sbd','sb-d','sd','sw-d']
                        if sum(a1) - sum(a2) > 0.3  :
                            count += 1
                            if 's' in lines[index].split("|")[3].strip(" "):
                                pos += 1
                            else:
                                print("Predicted wrong Sentence : " + lines[index] + " Predicted Label : sbd")

                        if sum(a2) - sum(a1) > 0.2 :
                            count += 1
                            if 'i' in lines[index].split("|")[3].strip(" ").lower():
                                pos += 1
                            else:
                                print("Predicted wrong Sentence : " + lines[index] + " Predicted Label : i")
        print(pos/count)   
        
    def load_model1(self, model_name):
            logger.info('Loading model: %s', model_name)

            model = MODELS[model_name.split('_')[0]]
            model_config = init_config(model_name.split('_')[0])

            self.model = model(self.max_doc_len, self.vocabulary_size, self.main_config, model_config)
            saver = tf.train.Saver()
            last_checkpoint = tf.train.latest_checkpoint('{}/{}'.format(self.model_dir, model_name))
            saver.restore(self.session, last_checkpoint)
            logger.info('Loaded model from: %s', last_checkpoint)
            saver.save(self.session, 'my_test_model',global_step=1000)
            
            f = open('./2020.txt','r')
            print(f.name)
            lines =f.readlines()
            scan_range = 5
            topics = []
            newLines = []
            for index, utterance in enumerate(lines):
                if index < scan_range or index >= len(lines) - scan_range:
                    continue
                
                if len(lines[index].split("|")[1].split()) > 3:
                    a1 = []
                    a2 = []

                    for i in range(-scan_range,0):
                        if len((lines[index+i]).split("|")[1].split()) > 3:
                            a1.append(self.predict(lines[index].split("|")[1], lines[index+i].split("|")[1]))
                        else:
                            continue

                    for i in range(1,scan_range+1):
                        if len((lines[index+i]).split("|")[1].split()) > 3:
                            a2.append(self.predict(lines[index].split("|")[1], lines[index+i].split("|")[1]))
                        else:
                            continue
            
                    maxVal = -10000000
                    indexTopic = -1
                    if sum(a1) - sum(a2) > 0.2:
                        print("a1,a2",sum(a1),sum(a2))
                        print(topics)
                        
                        if len(topics) < 1:
                            topics.append(index)
                        else:
                            for x in topics:
                                temp = self.predict(lines[index].split("|")[1],lines[x].split("|")[1])
                                if temp > maxVal:
                                    maxVal = temp
                                    indexTopic = x
                            if maxVal > 0.8:
                                print("Subdialogue of \n",lines[index],lines[indexTopic])
                                print(maxVal)
                    if sum(a2) - sum(a1) > 0.9:
                        topics.append(index)
                        print("Initiation : ", lines[index])

In [ ]:
gui = MultiheadSiameseNetGuiDemo()


INFO:tensorflow:Reading main configuration.
INFO:tensorflow:Loading model: rnn_64
INFO:tensorflow:Reading configuration for rnn model.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model_dir/rnn_64/model-6380
INFO:tensorflow:Loaded model from: model_dir/rnn_64/model-6380
./2020.txt
Instructions for updating:
Please use tensorflow/transform or tf.data.
a1,a2 2.0399999916553497 1.7300000190734863
[]
a1,a2 1.4500000029802322 0.8700000029057264
[7]
a1,a2 1.570000022649765 